In [2]:
import numpy as np
import pandas as pd
import os
import random

import tensorflow as tf
print(tf. __version__)
from tensorflow.keras.optimizers.legacy import Adam
import tensorflow_probability as tfp
from networks import ActorCriticNetwork
from NetworkEnv import NetworkEnv as Ne
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

2.14.0


In [3]:
train = []                            ## CREATING TRAIN SET OF 1000 GAMES
for i in range(1000):
  q1 = np.random.randint(2,6, 7)
  q2 = np.random.randint(1,6,7)
  q3 = np.random.randint(1,6,7)
  q4= np.random.randint(1,6,7)
  q5= np.random.randint(1,5,7)

  z = [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
  for i in range(len(q2)):
    if q2[i]==2:
      q2[i]+=1
  for i in range(len(q3)):
    if q3[i]==3:
      q3[i]+=1
  for i in range(len(q4)):
    if q4[i]==4:
      q4[i]+=1


  data = np.array([q1,q2,q3,q4,q5]).T
  data = np.vstack([data,z])
  df = pd.DataFrame(data, columns=["N1Q","N2Q","N3Q","N4Q","N5Q"])
  train.append(df)

print(len(train))



1000


In [4]:
def preprocess(new_state):            ## Preprocess state to turn into one hot state to be fed to Network
  q1 = new_state[:,0]
  q2 = new_state[:,1]
  q3 = new_state[:,2]
  q4 = new_state[:,3]
  q5 = new_state[:,4]
  q1 =np.array(q1)
  ohq1 = np.zeros((q1.shape[0], 6))
  ohq1[np.arange(q1.size),q1]=1

  q2 =np.array(q2)
  ohq2 = np.zeros((q2.shape[0], 6))
  ohq2[np.arange(q2.size),q2]=1
  

  q3 =np.array(q3)
  ohq3 = np.zeros((q3.shape[0], 6))
  ohq3[np.arange(q3.size),q3]=1

  q4 =np.array(q4)
  ohq4 = np.zeros((q4.shape[0], 6))
  ohq4[np.arange(q4.size),q4]=1

  q5 =np.array(q5)
  ohq5 = np.zeros((q5.shape[0], 6))
  ohq5[np.arange(q5.size),q5]=1

  ns = np.dstack((ohq1,ohq2,ohq3,ohq4,ohq5))
  return ns

In [5]:
class Agent:
    def __init__(self, env_name, alpha=0.00025, gamma=0.99, n_actions=[4,4,4,4,4]):
        self.gamma = gamma
        self.action_space = n_actions
        self.action = None
        self.actor_critic = ActorCriticNetwork(action_space=n_actions)
        self.env = Ne(env_name)



        self.actor_critic.compile(optimizer=Adam(learning_rate=alpha))

    def choose_action(self, observation):
        state = tf.convert_to_tensor([observation])
        _, probs = self.actor_critic(state)
        action_probabilities = tfp.distributions.Categorical(probs=probs)
        action = action_probabilities.sample()
        log_prob = action_probabilities.log_prob(action)
        self.action = action
        print(action)

        return action.numpy()


    def save_models(self):
        print('... saving models ...')
        self.actor_critic.save_weights(self.actor_critic.checkpoint_file)

    def load_models(self):
        print('... loading models ...')
        self.actor_critic.load_weights(self.actor_critic.checkpoint_file)

    def reset(self,df):
        self.env.reset(df)
        state = self.env.state_observation
        done = self.env.done
        reward = self.env.reward
        return state, done,reward





    def learn(self, state, reward, state_, done):
        state = tf.convert_to_tensor([state], dtype=tf.float32)
        state_ = tf.convert_to_tensor([state_], dtype=tf.float32)
        reward = tf.convert_to_tensor(reward, dtype=tf.float32)  # not fed to NN
        with tf.GradientTape(persistent=True) as tape:
            state_value, probs = self.actor_critic(state)
            state_value_, _ = self.actor_critic(state_)
            state_value = tf.squeeze(state_value)
            state_value_ = tf.squeeze(state_value_)

            action_probs = tfp.distributions.Categorical(probs=probs)
            log_prob = action_probs.log_prob(self.action)

            delta = reward + self.gamma * state_value_ * (1 - int(done)) - state_value
            actor_loss = -log_prob * delta
            critic_loss = delta ** 2
            total_loss = actor_loss + critic_loss

        gradient = tape.gradient(total_loss, self.actor_critic.trainable_variables)
        self.actor_critic.optimizer.apply_gradients(zip(
            gradient, self.actor_critic.trainable_variables))
        

    def run(self, df):
        # uncomment this line and do a mkdir tmp && mkdir video if you want to
        # record video of the agent playing the game.
        # env = wrappers.Monitor(env, 'tmp/video', video_callable=lambda episode_id: True, force=True)
        filename = 'results.png'

        figure_file = 'plots/' + filename

        best_score = -1000
        score_history = []
        load_checkpoint = False

        if load_checkpoint:
            self.load_models()

        for i in df:
            observation,done, score = self.reset(i)
            count = 0

            while not done:
                ohtob = preprocess(observation)
                action = self.choose_action(ohtob)
                observation_, reward, done, info = self.env.step(action, observation)
                score += reward
                ohtob_ = preprocess(observation_)
                print(score)
                print(action)
                print(observation)
                if not load_checkpoint:
                    self.learn(ohtob, reward, ohtob_, done)
                observation = observation_
                score_history.append(score)
                avg_score = np.mean(score_history[-100:])
                print(done)
                count +=1
                if count >200:
                  done = True
                  score -= 500
                
            

            
            if avg_score > best_score:
                best_score = avg_score
                if not load_checkpoint:
                    self.save_models()

            #print('episode ', i, 'score %.1f' % score, 'avg_score %.1f' % avg_score)

        if not load_checkpoint:
          N = len(score_history)
          running_avg = np.empty(N)
          for t in range(N):
            running_avg[t] = np.mean(scores[max(0, t - window):(t + 1)])
            if x is None:
              x = [i for i in range(N)]
            plt.ylabel('Score')
            plt.xlabel('Game')
            plt.plot(x, running_avg)
            plt.savefig(filename)

    def test(self, Model_name,df):
        self.load(Model_name)
        for e in range(100):
          state = self.reset(df)
          done = False
          score = 0
          while not done:
            print(done)
            one_hot_state = preprocess(state)
            one_hot_state = tf.convert_to_tensor([one_hot_state]) 
            prediction = self.Actor.predict(one_hot_state)
            action = np.zeros(5)
            count = 0
            for i in prediction:
              for j in i:
                action[count] = np.argmax(j)
                count +=1
            state, reward, done, _ = self.step(action)
            score += reward
            print(score)
 
          if done:
            print("episode: {}/{}, score: {}".format(e, self.EPISODES, score))
            print(average)
            break
        
      # close environemnt when finish training


In [ ]:
    env_name = ''
    agent = Agent(env_name)
    #agent.run(train)                      ## Running train sequence for agent on train set     

In [ ]:
agent.test("/content/Models/Nettest1_PG_1.h5",df)

episode: 0/2000, score: -3753


NameError: ignored

In [ ]:
x = [i + 1 for i in range(1000)]
plotLearning(x, self.score_history, figure_file)

NameError: ignored

In [6]:
print(train[999])

   N1Q  N2Q  N3Q  N4Q  N5Q
0    3    5    5    5    3
1    5    4    2    1    3
2    2    3    2    2    3
3    4    5    1    5    3
4    4    3    4    3    1
5    4    1    4    3    4
6    5    4    2    5    2
7    0    0    0    0    0
8    0    0    0    0    0
9    0    0    0    0    0


In [ ]:
agent.load_models()

... loading models ...


In [ ]:
print(agent.score_history)

AttributeError: ignored

In [ ]:
agent.run(df)

AttributeError: ignored